In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import os

# Load the GEBCO dataset
ds = xr.open_dataset('C:/Users/18479/OneDrive/Desktop/Code Turtle Project/gebco_08.nc')

##MAKE THIS BETTER VIA INTERPOLATION (NOT JUST THE CLOSEST GRIDPOINT)
def get_bathymetry(lat,lon,ds):
    # Get spacing and range from the dataset attributes
    spacing = ds.spacing.values  # This will give you the spacing array,[0.00833333, 0.00833333]
    x_range = ds.x_range.values  # This is the longitude range, [-180.0, 180.0]
    y_range = ds.y_range.values  # This is the latitude range, [-90.0, 90.0]

    # Calculate the index for latitude (y) and longitude (x)
    lat_idx = int((y_range[1] - lat) / spacing[1])  # Reverse the calculation for latitude
    lon_idx = int((lon - x_range[0]) / spacing[0])  # Longitude corresponds to x_range

    # Convert 2D indices to 1D index
    z_index = lat_idx * 43200  + lon_idx
    #z_index = lat_idx  + lon_idx * 21600
    # Get bathymetry value
    bathymetry = ds.z[z_index].values
    return bathymetry
    print(f"Latitude Index: {lat_idx}, Longitude Index: {lon_idx}")
    print(f"Bathymetry Value: {bathymetry_value}")
    return bathymetry_value
def addBath(file_path):
    """Add bathymetry data to a CSV file."""
    df = pd.read_csv(file_path)
    df['bathymetry'] = df.apply(lambda row: get_bathymetry(row['latitude'], row['longitude'], ds), axis=1)
    return df

# Example usage
TRACK_FILE = os.getenv('TRACK_FILE', 'path_to_track_file.csv')
new_df = addBath(TRACK_FILE)
print(new_df.head())

              datetime  latitude  longitude   deltaT  deltaLat  deltaLon  \
0  2011-09-15 08:55:50 -21.17789   55.26960      0.0   0.00000   0.00000   
1  2011-09-15 09:02:06 -21.17185   55.24230    376.0   0.00604  -0.02730   
2  2011-09-15 09:43:17 -21.17607   55.25989   2471.0  -0.00422   0.01759   
3  2011-09-15 13:40:38 -21.20071   55.27747  14241.0  -0.02464   0.01758   
4  2011-09-15 15:25:58 -21.18690   55.25729   6320.0   0.01381  -0.02018   

  bathymetry  
0      -78.0  
1     -216.0  
2     -116.0  
3        1.0  
4     -291.0  


In [ ]:
import MagneticModel as magmod
import pandas as pd
import numpy as np
import pickle
import os

MAGMODEL_PATH = os.getenv('MAGMODEL_PATH', 'path_to_magmodel_file.pkl')
with open(MAGMODEL_PATH, 'rb') as file:
    magmodel = pickle.load(file)

def addIntInc(file_path):
    df = pd.read_csv(file_path)
    df['Intensity'] = np.nan
    df['Inclination']=np.nan
    for i in range(0, len(df)):
        _, _, _, _, _, df.loc[i, 'Intensity'],df.loc[i, 'Inclination']=magmodel.evaluate_model(df.loc[i, 'latitude'], df.loc[i, 'longitude'])
    return df

# Example usage
TRACK_FILE = os.getenv('TRACK_FILE', 'path_to_track_file.csv')
new_df = addIntInc(TRACK_FILE)
print(new_df.head())

              datetime  latitude  longitude   deltaT  deltaLat  deltaLon  \
0  2011-09-15 08:55:50 -21.17789   55.26960      0.0   0.00000   0.00000   
1  2011-09-15 09:02:06 -21.17185   55.24230    376.0   0.00604  -0.02730   
2  2011-09-15 09:43:17 -21.17607   55.25989   2471.0  -0.00422   0.01759   
3  2011-09-15 13:40:38 -21.20071   55.27747  14241.0  -0.02464   0.01758   
4  2011-09-15 15:25:58 -21.18690   55.25729   6320.0   0.01381  -0.02018   

   Intensity  Inclination  
0 -54.222535    38.318013  
1 -54.214994    38.304775  
2 -54.220206    38.313337  
3 -54.247424    38.323929  
4 -54.231782    38.313210  


In [ ]:
import os
import pandas as pd
import numpy as np

def add_scrambled_path(df, pathnum, ds, magmodel):
    # Starting location and time from the DataFrame
    start_lat, start_lon, start_time = df['latitude'].iloc[0], df['longitude'].iloc[0], pd.to_datetime(df['datetime'].iloc[0])

    # Shuffle delta columns as rows
    scrambled_df = df[['deltaT', 'deltaLat', 'deltaLon']].sample(frac=1).reset_index(drop=True)

    # Initialize lists to store path coordinates and timestamps
    lats = [start_lat]
    lons = [start_lon]
    times = [start_time]

    # Generate path using scrambled delta values
    for _, row in scrambled_df.iterrows():
        # Compute new position
        new_lat = lats[-1] + row['deltaLat']
        new_lon = lons[-1] + row['deltaLon']
        new_time = times[-1] + pd.to_timedelta(row['deltaT'], unit='s')

        # Append to path lists
        lats.append(new_lat)
        lons.append(new_lon)
        times.append(new_time)


    # Prepare new columns for the scrambled path
    scrambled_path_data = {
        f"lat_fakepath_{pathnum}": lats[1:],
        f"lon_fakepath_{pathnum}": lons[1:],
        f"time_fakepath_{pathnum}": times[1:],
        f"bat_fakepath_{pathnum}": [
            get_bathymetry(lat, lon, ds) for lat, lon in zip(lats[1:], lons[1:])
        ],
        f"int_fakepath_{pathnum}": [np.nan] * len(lats[1:]),
        f"inc_fakepath_{pathnum}": [np.nan] * len(lats[1:]),
    }

    # Apply magnetic model evaluation
    for i in range(len(scrambled_path_data[f"lat_fakepath_{pathnum}"])):
        lat = scrambled_path_data[f"lat_fakepath_{pathnum}"][i]
        lon = scrambled_path_data[f"lon_fakepath_{pathnum}"][i]
        _, _, _, _, _, scrambled_path_data[f"int_fakepath_{pathnum}"][i], scrambled_path_data[f"inc_fakepath_{pathnum}"][i] = magmodel.evaluate_model(lat, lon)

    # Append the new columns to the DataFrame
    new_columns_df = pd.DataFrame(scrambled_path_data)
    return pd.concat([df, new_columns_df], axis=1)


# Directory containing CSV files
TRACKS_DIRECTORY = os.getenv('TRACKS_DIRECTORY', 'path_to_tracks_directory')

for filename in os.listdir(TRACKS_DIRECTORY):
    if filename.endswith(".csv") and filename.startswith("T"):  # Process only relevant CSV files
        file_path = os.path.join(TRACKS_DIRECTORY, filename)

        # Load the real path
        df = pd.read_csv(file_path, delimiter=',')
        print(df.head())

        for i in range(101):
            df = add_scrambled_path(df, str(i), ds, magmodel)
            print("made a new fake path")

        # Save a new CSV
        output_file_path = os.path.join(TRACKS_DIRECTORY, f"{os.path.splitext(filename)[0]}_fakepathsadded.csv")
        df.to_csv(output_file_path, index=False)

        print(f"Processed and saved: {output_file_path}")


              datetime  latitude  longitude   deltaT  deltaLat  deltaLon
0  2011-09-15 08:55:50 -21.17789   55.26960      0.0   0.00000   0.00000
1  2011-09-15 09:02:06 -21.17185   55.24230    376.0   0.00604  -0.02730
2  2011-09-15 09:43:17 -21.17607   55.25989   2471.0  -0.00422   0.01759
3  2011-09-15 13:40:38 -21.20071   55.27747  14241.0  -0.02464   0.01758
4  2011-09-15 15:25:58 -21.18690   55.25729   6320.0   0.01381  -0.02018
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake path
made a new fake 

In [ ]:
import os
import pandas as pd

# Directory containing .si files
FAKE_PATHS_DIRECTORY = os.getenv('FAKE_PATHS_DIRECTORY', 'path_to_fake_paths_directory')

# Loop through each file in the directory
for filename in os.listdir(FAKE_PATHS_DIRECTORY):
    file_path = os.path.join(FAKE_PATHS_DIRECTORY, filename)
    df = pd.read_csv(file_path)
    df.insert(3, 'inc_realpath',  np.nan)
    df.insert(3,'int_realpath',  np.nan)
    for i in range(0, len(df)):
        _, _, _, _, _, df.loc[i, 'Intensity'],df.loc[i, 'Inclination']=magmodel.evaluate_model(df.loc[i, 'latitude'], df.loc[i, 'longitude'])
    df.insert(3, 'bat_realpath', df.apply(lambda row: get_bathymetry(row['latitude'], row['longitude'], ds), axis=1))
    df = df.drop(columns=['deltaT', 'deltaLat','deltaLon'])
    df.to_csv(file_path, index=False)


In [ ]:
FAKE_PATHS_DIRECTORY = os.getenv('FAKE_PATHS_DIRECTORY', 'path_to_fake_paths_directory')
for filename in os.listdir(FAKE_PATHS_DIRECTORY):
    file_path = os.path.join(FAKE_PATHS_DIRECTORY, filename)
    df = pd.read_csv(file_path)
    print(file_path)
    print(df.isnull().values.any())
    if 'Intensity' in df.columns:
        df["int_realpath"]=df['Intensity']
        df["inc_realpath"]=df['Inclination']
        df = df.drop(columns=['Intensity', 'Inclination'])
        df.to_csv(file_path, index=False)

C:/Users/18479/OneDrive/Desktop/Code Turtle Project/FakePathsAdded\combined_turtle_data_long.csv
True
C:/Users/18479/OneDrive/Desktop/Code Turtle Project/FakePathsAdded\T53434_cleaned_fakepathsadded.csv
False
C:/Users/18479/OneDrive/Desktop/Code Turtle Project/FakePathsAdded\T53509_cleaned_fakepathsadded.csv
False
C:/Users/18479/OneDrive/Desktop/Code Turtle Project/FakePathsAdded\T57684_cleaned_fakepathsadded.csv
False
C:/Users/18479/OneDrive/Desktop/Code Turtle Project/FakePathsAdded\T66818_cleaned_fakepathsadded.csv
False
C:/Users/18479/OneDrive/Desktop/Code Turtle Project/FakePathsAdded\T66819_cleaned_fakepathsadded.csv
False
C:/Users/18479/OneDrive/Desktop/Code Turtle Project/FakePathsAdded\T66821_cleaned_fakepathsadded.csv
False
C:/Users/18479/OneDrive/Desktop/Code Turtle Project/FakePathsAdded\T66823_cleaned_fakepathsadded.csv
False
C:/Users/18479/OneDrive/Desktop/Code Turtle Project/FakePathsAdded\T66837_cleaned_fakepathsadded.csv
False
C:/Users/18479/OneDrive/Desktop/Code Turtl

In [30]:
df.isnull().values.any()

False

In [28]:
print(df.columns)

Index(['datetime', 'latitude', 'longitude', 'bat_realpath', 'int_realpath',
       'inc_realpath', 'lat_fakepath_0', 'lon_fakepath_0', 'time_fakepath_0',
       'bat_fakepath_0',
       ...
       'time_fakepath_99', 'bat_fakepath_99', 'int_fakepath_99',
       'inc_fakepath_99', 'lat_fakepath_100', 'lon_fakepath_100',
       'time_fakepath_100', 'bat_fakepath_100', 'int_fakepath_100',
       'inc_fakepath_100'],
      dtype='object', length=612)
